In [1]:
# Frequentist Neural Network for Classification

# Required Libraries
import argparse
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import os
import time


In [3]:
class SimpleFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)  # Dropout after first layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)  # Dropout applied here
        out = self.fc2(out)
        return out


In [106]:
# # uncomment for Cancer Dataset

# def load_and_preprocess_data(file_path, test_size=0.3, random_state=42):
#     df = pd.read_csv(file_path)
#     print("Loaded dataset with shape:", df.shape)

#     # Define target and feature columns correctly
#     target_col = "diagnosis"
#     feature_cols = df.columns[2:]  # Exclude 'id' (first column) and take features from column 3 onward

#     # Extract features (X) and target labels (y)
#     X = df[feature_cols].values
#     y = df[target_col].values

#     # Encode 'diagnosis' column ('M' -> 1, 'B' -> 0)
#     le = LabelEncoder()
#     y = le.fit_transform(y)  # Converts "M"/"B" to 1/0

#     # Standardize features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     # Check for NaNs and replace if necessary
#     if np.isnan(X_scaled).any():
#         print("⚠️ Warning: NaNs found in dataset! Replacing with zeros.")
#         X_scaled = np.nan_to_num(X_scaled)

#     # Print class distribution
#     unique, counts = np.unique(y, return_counts=True)
#     print("Class Distribution:", dict(zip(unique, counts)))

#     # Ensure valid test size
#     min_class_samples = min(counts)
#     test_size = min(0.3, max(0.1, (min_class_samples - 1) / len(y)))

#     # Perform train-test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_scaled, y, test_size=test_size, random_state=random_state, stratify=y if min_class_samples > 1 else None
#     )

#     # Convert to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#     y_train_tensor = torch.tensor(y_train, dtype=torch.long)
#     X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

#     input_dim = X_train_tensor.shape[1]
#     output_dim = len(np.unique(y))  # Should be 2 (Benign/Malignant)

#     return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, input_dim, output_dim


In [ ]:
# # uncomment for Diabetes datset
# def load_and_preprocess_data(file_path, test_size=0.3, random_state=42):
#     # Load dataset
#     df = pd.read_csv(file_path)
#     print("Loaded dataset with shape:", df.shape)

#     # Define target and feature columns
#     target_col = "Outcome"
#     feature_cols = df.columns[:-1]  # Use all columns except 'Outcome' as features

#     # Extract features (X) and target labels (y)
#     X = df[feature_cols].values
#     y = df[target_col].values  # Already numeric (0 or 1), no encoding needed

#     # Check for missing values and replace NaNs with column means
#     df.replace(0, np.nan, inplace=True)  # Convert zeros to NaN (common in medical datasets)
#     df.fillna(df.mean(), inplace=True)   # Replace NaNs with mean values

#     # Standardize features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     # Print class distribution
#     unique, counts = np.unique(y, return_counts=True)
#     print("Class Distribution:", dict(zip(unique, counts)))

#     # Ensure valid test size
#     min_class_samples = min(counts)
#     test_size = min(0.3, max(0.1, (min_class_samples - 1) / len(y)))

#     # Perform train-test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_scaled, y, test_size=test_size, random_state=random_state, stratify=y if min_class_samples > 1 else None
#     )

#     # Convert to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#     y_train_tensor = torch.tensor(y_train, dtype=torch.long)
#     X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

#     input_dim = X_train_tensor.shape[1]
#     output_dim = len(np.unique(y))  # Should be 2 (Diabetic/Non-Diabetic)

#     return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, input_dim, output_dim


In [108]:
# uncomment for heart_statlog_cleveland_hungary_final Dataset
# def load_and_preprocess_data(file_path, test_size=0.3, random_state=42):
#     # Load dataset
#     df = pd.read_csv(file_path)
#     print("Loaded dataset with shape:", df.shape)

#     # Define target and feature columns
#     target_col = "target"
#     feature_cols = df.columns[:-1]  # Use all columns except 'target' as features

#     # Extract features (X) and target labels (y)
#     X = df[feature_cols]
#     y = df[target_col].values  

#     # Identify categorical columns (if any)
#     categorical_cols = ["sex", "chest pain type", "fasting blood sugar", "resting ecg", "exercise angina", "ST slope"]
    
#     # Convert categorical columns to numerical using one-hot encoding
#     X = pd.get_dummies(X, columns=categorical_cols)

#     # Handle missing values
#     X.replace(0, np.nan, inplace=True)  # Convert zeros to NaN (common in medical datasets)
#     X.fillna(X.mean(), inplace=True)    # Replace NaNs with column means

#     # Standardize numerical features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     # Check class distribution
#     unique, counts = np.unique(y, return_counts=True)
#     print("Class Distribution:", dict(zip(unique, counts)))

#     # Ensure valid test size
#     min_class_samples = min(counts)
#     test_size = min(0.3, max(0.1, (min_class_samples - 1) / len(y)))

#     # Perform train-test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_scaled, y, test_size=test_size, random_state=random_state, stratify=y if min_class_samples > 1 else None
#     )

#     # Convert to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#     y_train_tensor = torch.tensor(y_train, dtype=torch.long)
#     X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

#     input_dim = X_train_tensor.shape[1]
#     output_dim = len(np.unique(y))  # Should be 2 if binary classification, else check for multi-class

#     return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, input_dim, output_dim


In [109]:
# uncomment for heart Dataset
# def load_and_preprocess_data(file_path, test_size=0.3, random_state=42):
#     # Load dataset
#     df = pd.read_csv(file_path)
#     print("Loaded dataset with shape:", df.shape)

#     # Define target and feature columns
#     target_col = "target"
#     feature_cols = df.columns[:-1]  # Use all columns except 'target' as features

#     # Extract features (X) and target labels (y)
#     X = df[feature_cols]
#     y = df[target_col].values  

#     # Identify categorical columns (if any)
#     categorical_cols = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]
    
#     # Convert categorical columns to numerical using one-hot encoding
#     X = pd.get_dummies(X, columns=categorical_cols)

#     # Handle missing values
#     X.replace(0, np.nan, inplace=True)  # Convert zeros to NaN (common in medical datasets)
#     X.fillna(X.mean(), inplace=True)    # Replace NaNs with column means

#     # Standardize numerical features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     # Check class distribution
#     unique, counts = np.unique(y, return_counts=True)
#     print("Class Distribution:", dict(zip(unique, counts)))

#     # Ensure valid test size
#     min_class_samples = min(counts)
#     test_size = min(0.3, max(0.1, (min_class_samples - 1) / len(y)))

#     # Perform train-test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_scaled, y, test_size=test_size, random_state=random_state, stratify=y if min_class_samples > 1 else None
#     )

#     # Convert to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#     y_train_tensor = torch.tensor(y_train, dtype=torch.long)
#     X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

#     input_dim = X_train_tensor.shape[1]
#     output_dim = len(np.unique(y))  # Should be 2 if binary classification, else check for multi-class

#     return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, input_dim, output_dim


In [110]:

# # uncomment for hepatitis Dataset
# def load_and_preprocess_data(file_path, test_size=0.3, random_state=42):
#     # Load dataset, replacing "?" with NaN
#     df = pd.read_csv(file_path, na_values="?")
#     print("Loaded dataset with shape:", df.shape)

#     # Define target and feature columns
#     target_col = "out_class"
#     feature_cols = df.columns[1:]  # Use all columns except 'out_class' as features

#     # Extract features (X) and target labels (y)
#     X = df[feature_cols]
#     y = df[target_col].values  

#     # 🔍 Convert Target Labels (out_class)
#     print("\nConverting `out_class`: 2 → 1 (Survived), 1 → 0 (Died)")
#     df["out_class"] = df["out_class"].replace({2: 1, 1: 0})
#     y = df["out_class"].values

#     # 🔍 Convert Binary Features (1 → 0 "No", 2 → 1 "Yes")
#     binary_cols = ["sex", "steroid", "antivirals", "fatigue", "malaise", "anorexia",
#                    "liver_big", "liver_firm", "spleen_palable", "spiders", "ascites",
#                    "varices", "histology"]

#     print("\nConverting binary categorical columns: 1 → 0 (No), 2 → 1 (Yes)")
#     df[binary_cols] = df[binary_cols].replace({1: 0, 2: 1})

#     # 🔍 Handle Missing Values
#     print("\nReplacing missing values with column means...")
#     df.fillna(df.mean(), inplace=True)

#     # Standardize numerical features
#     numerical_cols = ["age", "bilirubin", "alk_phosphate", "sgot", "albumin", "protime"]
#     scaler = StandardScaler()
#     df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

#     # Extract processed feature matrix
#     X = df[feature_cols].values

#     # 🔍 Check Class Distribution
#     unique, counts = np.unique(y, return_counts=True)
#     print("\nFinal Class Distribution:", dict(zip(unique, counts)))

#     # Ensure valid test size
#     min_class_samples = min(counts)
#     test_size = min(0.3, max(0.1, (min_class_samples - 1) / len(y)))

#     # Perform train-test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=random_state, stratify=y if min_class_samples > 1 else None
#     )

#     # Convert to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
#     y_train_tensor = torch.tensor(y_train, dtype=torch.long)
#     X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

#     input_dim = X_train_tensor.shape[1]
#     output_dim = len(np.unique(y))  # Should be 2 for binary classification

#     return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, input_dim, output_dim

In [111]:
def train_model(model, X_train, y_train, epochs=100, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    outputs = model(X_train)  # No softmax!
    loss = criterion(outputs, y_train)

    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")



In [112]:
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)  # Get class with highest probability
        accuracy = (predicted == y_test).float().mean().item()
    print(f"Test Accuracy: {accuracy*100:.2f}%")


In [ ]:
def main():
    # Define the datasets folder path relative to this script
    base_path = os.path.join(os.getcwd(), "DataSets")
    
    # Specify the CSV file to use. Change this to run a different dataset.
    # dataset_file = "cancer.csv"
    dataset_file = "diabetes.csv"
    # dataset_file = "heart_statlog_cleveland_hungary_final.csv"
    # dataset_file = "heart.csv"
    # dataset_file = "hepatitis.csv"


    file_path = os.path.join(base_path, dataset_file)
    
    print("Using dataset:", file_path)
    
    # Load dataset
    # X_train, X_test, y_train, y_test, input_dim, output_dim = load_and_preprocess_data(file_path)

    print(f"Input dimension: {input_dim}, Output dimension: {output_dim}")

    # Instantiate the model
    model = SimpleFNN(input_dim, hidden_dim=3, output_dim=output_dim)

    # Print class distribution
    unique, counts = np.unique(y_train.numpy(), return_counts=True)
    print("Train Class Distribution:", dict(zip(unique, counts)))

    unique, counts = np.unique(y_test.numpy(), return_counts=True)
    print("Test Class Distribution:", dict(zip(unique, counts)))

    model.train()  # set model to training mode
    start_train = time.time()
    print("Training model...")
    train_model(model, X_train, y_train, epochs=1000, lr=0.001)
    end_train = time.time()
    train_time = end_train - start_train
    print("Training time for Freqtist NN: {:.4f} seconds".format(train_time))

    
    start_infer = time.time()
    print("Evaluating model...")
    evaluate_model(model, X_test, y_test)
    end_infer = time.time()
    infer_time = end_infer - start_infer
    print("Inference time for Freqtist NN: {:.4f} seconds".format(infer_time))

    



In [114]:
if __name__ == "__main__":
    main()

Using dataset: /workspaces/paviabera/Desktop/Archive BVNN/code/DataSets/diabetes.csv
Loaded dataset with shape: (768, 9)
Class Distribution: {np.int64(0): np.int64(500), np.int64(1): np.int64(268)}
Input dimension: 8, Output dimension: 2
Train Class Distribution: {np.int64(0): np.int64(350), np.int64(1): np.int64(187)}
Test Class Distribution: {np.int64(0): np.int64(150), np.int64(1): np.int64(81)}
Training model...


Epoch [10/1000], Loss: 0.7139
Epoch [20/1000], Loss: 0.7026
Epoch [30/1000], Loss: 0.6939
Epoch [40/1000], Loss: 0.6894
Epoch [50/1000], Loss: 0.6798
Epoch [60/1000], Loss: 0.6749
Epoch [70/1000], Loss: 0.6680
Epoch [80/1000], Loss: 0.6559
Epoch [90/1000], Loss: 0.6583
Epoch [100/1000], Loss: 0.6548
Epoch [110/1000], Loss: 0.6465
Epoch [120/1000], Loss: 0.6461
Epoch [130/1000], Loss: 0.6438
Epoch [140/1000], Loss: 0.6334
Epoch [150/1000], Loss: 0.6293
Epoch [160/1000], Loss: 0.6234
Epoch [170/1000], Loss: 0.6114
Epoch [180/1000], Loss: 0.6255
Epoch [190/1000], Loss: 0.6170
Epoch [200/1000], Loss: 0.6104
Epoch [210/1000], Loss: 0.6116
Epoch [220/1000], Loss: 0.6014
Epoch [230/1000], Loss: 0.6028
Epoch [240/1000], Loss: 0.6065
Epoch [250/1000], Loss: 0.6003
Epoch [260/1000], Loss: 0.5941
Epoch [270/1000], Loss: 0.5924
Epoch [280/1000], Loss: 0.6048
Epoch [290/1000], Loss: 0.6009
Epoch [300/1000], Loss: 0.5836
Epoch [310/1000], Loss: 0.5867
Epoch [320/1000], Loss: 0.5653
Epoch [330/1000],